In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%config InlineBackend.figure_format='retina'

____
# Load and clean data

In [ ]:
data_date = "XXX"
include_carmen = False

In [ ]:
data_path = "XXX"
dict_path = "XXX"
col_meta_path = "XXX"

In [ ]:
dict_df = pd.read_csv(dict_path)

def explain_col(col):
    temp = dict_df[dict_df["Variable / Field Name"] == col]
    print(temp)
    
dict_df

In [ ]:
col_meta = pd.read_excel(col_meta_path, sheet_name="columns_metadata")
col_meta

In [ ]:
data_df_raw = pd.read_csv(data_path, low_memory=False)
data_df_raw

Clean data:

In [ ]:
# Remove patient IDs that contain a letter or is length 5 or less
data_df = data_df_raw[
    ~data_df_raw['record_id'].str.contains('[a-zA-Z]', regex=True) & 
    (data_df_raw['record_id'].str.len() > 5)
].copy()

print(len(data_df_raw))
print(len(data_df))

In [ ]:
print(len(data_df.columns))

# Check for any columns in data_df that are not mentioned in col_meta
extra_columns = set(data_df.columns) - set(col_meta["column"])
if extra_columns:
    print("Warning: The following columns in data_df are not mentioned in col_meta:", extra_columns)

# Identify columns to delete
for col in col_meta["column"]:
    if col_meta.loc[col_meta["column"] == col, "delete_manual"].values[0] == 1:
        try:
            del data_df[col]
        except KeyError:
            print(f"Column {col} not found in data.")

print(len(data_df.columns))

In [ ]:
# Drop pathogen columns that won't be used for pathogen detection
all_path_cols =  col_meta.loc[col_meta["pathogen"].notnull(), "column"].tolist()
path_cols_to_keep = col_meta.loc[col_meta["pathogen_use_clean"].notnull(), "column"].tolist()
col_to_exclude = [col for col in all_path_cols if col not in path_cols_to_keep]

# Of the cols to exclude, identify columns that exist in data_df and those that are missing
existing_cols = [col for col in col_to_exclude if col in data_df.columns]
missing_cols = [col for col in col_to_exclude if col not in data_df.columns]

# Drop only existing columns
data_df = data_df.drop(columns=existing_cols)

# Print warning for missing columns
if missing_cols:
    print("The following columns were not found in the DataFrame (and were not dropped):")
    print(missing_cols)

print(len(data_df.columns))

In [ ]:
def clean_dataframe(df, nan_threshold, keep_cols):
    if keep_cols is None:
        keep_cols = []

    # Normalize to list of strings and ensure 'record_id' is preserved
    keep_cols = list(set(map(str, keep_cols)) | {'record_id'})

    original_cols = set(df.columns)

    # Drop columns with too many NaNs (unless excluded)
    threshold = nan_threshold * len(df)
    cols_to_keep = [col for col in df.columns if col in keep_cols or df[col].count() >= threshold]
    df = df[cols_to_keep]

    # Identify object/string columns to drop (excluding protected ones)
    text_columns = df.select_dtypes(include=['object', 'string']).columns
    str_cols = [col for col in text_columns if col not in keep_cols]

    # Drop those string columns
    df = df.drop(columns=str_cols)

    # Determine dropped columns
    final_cols = set(df.columns)
    dropped_cols = list(original_cols - final_cols)

    return df, str_cols, dropped_cols

In [ ]:
# Remove all columns with text values or >50% NaN (but exclude pathogen columns)
nan_threshold = 0.5

# Manually keep specially marked columns and pathogen diagnosis columns
keep_manual_list = list(col_meta[col_meta["keep_manual"] == 1]["column"].values) + path_cols_to_keep

keep_cols = col_meta.loc[col_meta["pathogen_use_clean"].notnull(), "column"].tolist() + keep_manual_list
data_df, dropped_str_cols, all_dropped_cols = clean_dataframe(
    data_df,
    nan_threshold=nan_threshold,
    keep_cols=keep_cols
)

print("Dropped text columns:", dropped_str_cols)
print("All dropped columns:", all_dropped_cols)

In [ ]:
print(len(data_df.columns))

___
# Add pathogen labels

In [ ]:
col_meta[col_meta["pathogen"].notnull()]["pathogen"].unique()

In [ ]:
pathogens = col_meta[col_meta["pathogen"].notnull()]["pathogen"].unique()
pathogens

In [ ]:
# Identify pathogen-related columns
pathogen_cols = col_meta.dropna(subset=["pathogen"])[["column", "pathogen"]]

# Filter columns that exist in data_df
valid_pathogen_cols = [col for col in pathogen_cols["column"] if col in data_df.columns]

# Exclude columns with string values and alert the user
excluded_cols = []
numeric_cols = []
for col in valid_pathogen_cols:
    if data_df[col].dtype == object:  # Checking for string-type columns
        pathogen = pathogen_cols.loc[pathogen_cols["column"] == col, "pathogen"].values[0]
        excluded_cols.append((col, pathogen))
    else:
        numeric_cols.append(col)

# Print excluded columns with pathogen information
if excluded_cols:
    print("The following columns were excluded due to containing string values:")
    for col, pathogen in excluded_cols:
        print(f"- Column: {col}, Pathogen: {pathogen}")

# Proceed only with numeric columns
pathogen_data = data_df[numeric_cols]

# Summarize counts (assuming categorical data; adjust if numeric)
summary_df = pathogen_data.apply(pd.Series.value_counts).fillna(0).astype(int)

# Create stacked bar plot if there are valid numeric columns
if not summary_df.empty:
    plt.figure(figsize=(12, 6))
    summary_df.T.plot(kind='bar', stacked=True, colormap="Reds", figsize=(12, 6))

    # Customize plot
    plt.xlabel("Pathogen Columns")
    plt.ylabel("Count")
    plt.title("Stacked Bar Plot of Pathogen-Related Columns in data_df")
    plt.legend(title="Values", bbox_to_anchor=(1, 1))

    plt.tight_layout()
    plt.show()
else:
    print("No numeric pathogen-related columns available for plotting.")

In [ ]:
list(dict_df[dict_df['Variable / Field Name']=='lassa_pcr']['Choices, Calculations, OR Slider Labels'])

### Create positive/negative labels for each pathogen and remove all pathogen-related columns

In [ ]:
# Create positive/negative labels for each pathogen and remove all pathogen-related columns from feature space
pos_neg_nums = {}
data_df_clean_pathogen = data_df.copy()
skipped_pathogens = []
for pathogen in pathogens:
    # Subset data for each pathogen based on col_meta
    pathogen_cols = col_meta[col_meta["pathogen"] == pathogen]["column"].values
    diagnosis_cols = col_meta[col_meta["pathogen_use_clean"] == 1]["column"].values

    if include_carmen:
        pathogen_diagnosis_cols = [item for item in pathogen_cols if item in diagnosis_cols]
    else:
        pathogen_diagnosis_cols = [item for item in pathogen_cols if item in diagnosis_cols and "carmen" not in item]

    if len(pathogen_diagnosis_cols) > 0:
        condition_1 = (data_df_clean_pathogen[pathogen_diagnosis_cols] == 1).any(axis=1)
        condition_2 = (data_df_clean_pathogen[pathogen_diagnosis_cols] == 2).any(axis=1)
    
        # Assigns labels: 0 = Negative, 1 = Positive, 2 = No result
        # Label is 1 if any relevant column equals 1
        # Label is 0 if any relevant column equals 2 (and none are 1)
        data_df_clean_pathogen[f"{pathogen}_label"] = np.select(
                [condition_1, condition_2], [1, 0], default=2
            )
    
        # Save number of confirmed positive and negative cases
        pos_num = len(data_df_clean_pathogen[data_df_clean_pathogen[f'{pathogen}_label'] == 1])
        neg_num = len(data_df_clean_pathogen[data_df_clean_pathogen[f'{pathogen}_label'] == 0])
        print(f"{pathogen} Pos: {pos_num}")
        print(f"{pathogen} Neg: {neg_num}")
        pos_neg_nums[pathogen] = [pos_num, neg_num]

    else:
        skipped_pathogens.append(pathogen)
        print(f"{pathogen} skipped because no diagnosis columns were found")

# Drop all original pathogen test columns from the feature space
all_pathogen_cols = col_meta[col_meta["pathogen"].notnull()]["column"].values
pathogen_cols_to_drop = [col for col in all_pathogen_cols if col in data_df_clean_pathogen.columns]

data_df_clean_pathogen = data_df_clean_pathogen.drop(columns=pathogen_cols_to_drop)

### Plot number of positive/negative per pathogen:

In [ ]:
# Extract pathogen names and their corresponding counts
path_df = pd.DataFrame()
path_df['pathogen'] = list(pos_neg_nums.keys())
path_df['pos_count'] = [pos_neg_nums[p][0] for p in pathogens if p not in skipped_pathogens]
path_df['neg_count'] = [pos_neg_nums[p][1] for p in pathogens if p not in skipped_pathogens]
path_df = path_df.sort_values(['pos_count', 'neg_count'], ascending=False) 

pathogens_plot = path_df['pathogen']
pos_counts = path_df['pos_count']
neg_counts = path_df['neg_count']

# Set bar width and positions
bar_width = 0.4
x = np.arange(len(pathogens_plot))

# Create bar plot
fig, ax = plt.subplots(figsize=(12, 5))

bars_pos = ax.bar(x - bar_width/2, pos_counts, bar_width, label="Positive", color="red")
bars_neg = ax.bar(x + bar_width/2, neg_counts, bar_width, label="Negative", color="lightgrey")

# Labeling
ax.set_xticks(x)
ax.set_xticklabels(pathogens_plot, rotation=30, ha="right")
ax.set_ylabel("Sample Count")
ax.set_title("# Confirmed Positive & Negative Samples per Pathogen")
# ax.legend(loc='upper left', bbox_to_anchor=(1, 1)) # legend outside plot
ax.legend(loc='upper right')
ax.margins(x=0.01)

# Add the number above each bar
for bar in bars_pos:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 200, f'{int(yval):,}', ha='center', va='bottom', rotation=90, fontsize=8)
for bar in bars_neg:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 200, f'{int(yval):,}', ha='center', va='bottom', rotation=90, fontsize=8)

ax.grid(True, axis='y',  color="lightgrey", ls="--", lw=1)
ax.set_axisbelow(True)

ax.set_ylim(top=15500)

fig.savefig("figures/1_sample_nums_per_pathogen.png", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
# Extract pathogen names and their corresponding counts, excluding pathogens with 0 confirmed positive AND 0 negative tests
pathogen_data = []
for p in pathogens:
    if p not in skipped_pathogens:
        pos_count = pos_neg_nums[p][0]
        neg_count = pos_neg_nums[p][1]
        # Only include if at least one positive or negative
        if (pos_count > 0) or (neg_count > 0):
            pathogen_data.append({'pathogen': p, 'pos_count': pos_count, 'neg_count': neg_count})

path_df = pd.DataFrame(pathogen_data)
path_df = path_df.sort_values(['pos_count', 'neg_count'], ascending=False)

pathogens_plot = path_df['pathogen']
pos_counts = path_df['pos_count']
neg_counts = path_df['neg_count']

# Set bar width and positions
bar_width = 0.4
x = np.arange(len(pathogens_plot))

# Create bar plot
fig, ax = plt.subplots(figsize=(12, 4.5))

bars_pos = ax.bar(x - bar_width/2, pos_counts, bar_width, label="Positive", color="red")
bars_neg = ax.bar(x + bar_width/2, neg_counts, bar_width, label="Negative", color="lightgrey")

# Labeling
ax.set_xticks(x)
ax.set_xticklabels(pathogens_plot, rotation=30, ha="right")
ax.set_ylabel("Sample Count")
ax.set_title("# Confirmed Positive & Negative Samples per Pathogen")
# ax.legend(loc='upper left', bbox_to_anchor=(1, 1)) # legend outside plot
ax.legend(loc='upper right')
ax.margins(x=0.01)

# Add the number above each bar
for bar in bars_pos:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 200, f'{int(yval):,}', ha='center', va='bottom', rotation=90, fontsize=8)
for bar in bars_neg:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 200, f'{int(yval):,}', ha='center', va='bottom', rotation=90, fontsize=8)

ax.grid(True, axis='y',  color="lightgrey", ls="--", lw=1)
ax.set_axisbelow(True)

ax.set_ylim(top=15500)

fig.savefig("figures/1_sample_nums_per_pathogen_short.png", dpi=300, bbox_inches="tight")

plt.show()

___

# Plot features

In [ ]:
def plot_histogram(data_df, pathogen, feature, symlog=False):
    fig, ax = plt.subplots(figsize=(10, 5))
    fontsize = 12

    data_df_temp = data_df.copy()

    if feature == "date_crf":
        data_df_temp[feature] = pd.to_datetime(data_df_temp[feature], errors='coerce')

    # Map numeric labels to readable categories
    label_col = f"{pathogen}_label"
    data_df_temp[label_col] = data_df_temp[label_col].map({1: "positive", 0: "negative", 2: "undefined"})

    # Define color palette
    palette = {"positive": "red", "negative": "black", "undefined": "grey"}

    # Plot stacked histogram
    sns.histplot(
        data=data_df_temp,
        x=feature,
        hue=label_col,
        bins=70,
        element='bars',
        multiple='stack',
        stat='count',  # Use 'proportion' for normalized values
        palette=palette,
        ax=ax,
        lw=0,
        alpha=0.7
    )

    # Axis labels and formatting
    ax.set_xlabel(feature, fontsize=fontsize)
    ax.set_ylabel("Sample count", fontsize=fontsize)
    ax.margins(x=0.01)
    ax.grid(True, axis='y', color="lightgrey", ls="--", lw=1)
    ax.set_axisbelow(True)

    if symlog:
        ax.set_yscale("symlog")

    if feature == "date_crf":
        # Format x-axis by month
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize-2)

    # Legend
    # ax.legend(title="Infection Status", fontsize=fontsize)
    plt.tight_layout()

    fig.savefig(f"figures/1_{pathogen}_{feature}_stacked.png", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
feature = "date_crf"
pathogen = "Malaria"

# Only plot recent samples
df_to_plot = data_df_clean_pathogen[
    pd.to_datetime(data_df_clean_pathogen[feature], errors='coerce') > pd.to_datetime("2022-01-01")
].copy()

plot_histogram(df_to_plot, pathogen, feature)

In [ ]:
feature = "temperature"
pathogen = "Malaria"

plot_histogram(data_df_clean_pathogen, pathogen, feature, symlog=True)

In [ ]:
def convert_to_celsius(temp):
    try:
        temp = float(temp)
    except:
        return np.nan

    converted = None

    # Celsius ×10 (common in clinical data)
    if 300 <= temp <= 450:
        converted = temp / 10

    # Normal Celsius range
    elif 30 <= temp <= 45:
        converted = temp

    # Fahrenheit range
    elif 80 <= temp <= 120:
        converted = (temp - 32) * 5 / 9

    # Kelvin range
    elif 305 <= temp <= 320:
        converted = temp - 273.15

    # Final validity check
    if converted is not None and 30 <= converted <= 45:
        return converted
    else:
        return np.nan

In [ ]:
data_df_clean_pathogen["temperature"] = data_df_clean_pathogen["temperature"].apply(convert_to_celsius)

In [ ]:
feature = "temperature"
pathogen = "Malaria"

plot_histogram(data_df_clean_pathogen, pathogen, feature, symlog=True)

___
# Save clean data

In [ ]:
import json

with open(f"XXX.json", "w") as f:
    json.dump(pos_neg_nums, f, indent=2)

In [ ]:
data_df_clean_pathogen.to_csv(f"XXX", index=False)

Record deleted columns:

In [ ]:
# Compare raw to cleaned data
# df1_cols = set(data_df_raw.columns)
df1_cols = set(col_meta['column'].values)
df2_cols = set(data_df_clean_pathogen.columns)

only_in_df1 = df1_cols - df2_cols
only_in_df2 = df2_cols - df1_cols

print("Columns only in raw data:")
print(sorted(only_in_df1))

print("\nColumns only in clean data:")
print(sorted(only_in_df2))

In [ ]:
# Set final_delete column to 1 if columns was deleted
col_meta.loc[col_meta['column'].isin(only_in_df1), 'final_delete'] = 1

In [ ]:
col_meta.to_csv("../data_small/pathogen_metadata.csv", index=False)

Note: Manually transfer the final_delete to Google sheet if changed